# Guardian - Iteration 13


## Import Packages

In [6]:
# load .env File in Environment
from dotenv import load_dotenv
# operating system interfaces - get Environment-variables
import os
# Connect to API
import requests
# create Pandas Data Frame
import pandas as pd
import datetime as dt
from functools import wraps
load_dotenv()

True

### Set Params

In [7]:
iteration = 13

content_url = "https://content.guardianapis.com/search"
all_content_params = {
    "page-size": "50",
    "page-size": "50",
    "show-fields": "all",
    "show-tags": "all",
    "from-date": "2017-03-01",
    "to-date": "2017-07-01",
    "order-by": "newest",
    "api-key": os.getenv("GUARDIAN_API_KEY2"),
}


In [8]:
def log_step(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        tic = dt.datetime.now()
        result = func(*args, **kwargs)
        time_taken = str(dt.datetime.now() - tic)
        print(f"{func.__name__}:\n shape={result.shape} took {time_taken}s\n")
        return result

    return wrapper

### Create DataFrame function


In [9]:
def get_df(url: str, params: dict):
        all_results = []
        current_page = 1
        total_pages = 1
        while current_page <= total_pages:
                tic = dt.datetime.now()
                params["page"] = current_page
                try:
                        r = requests.get(url, params)
                        all_results = all_results + r.json()["response"]["results"]
                        r.raise_for_status()
                except Exception as err:
                        SystemExit(err)
                if current_page == 1:
                        print("---- API STATUS ---- ")
                        print("status",  r.json()["response"]["status"])
                        total_pages = r.json()['response']['pages']
                        print("URL: ", r.url)
                        print("status",  r.json()["response"]["status"])
                        print("total",  r.json()["response"]["total"])
                        print("startIndex",  r.json()["response"]["startIndex"])
                        print("pageSize",  r.json()["response"]["pageSize"])
                        print("pages",  r.json()["response"]["pages"])
                        print("orderBy",  r.json()["response"]["orderBy"])
                        print("---- RUNTIME STATUS ---- ")

                time_taken = str(dt.datetime.now() - tic)
                print(f"({current_page}/{total_pages}) in {time_taken}s")
                        
                current_page += 1

        return pd.DataFrame(all_results)

### API Request

In [10]:
init_df = get_df(content_url, all_content_params)

---- API STATUS ---- 
status ok
URL:  https://content.guardianapis.com/search?page-size=50&show-fields=all&show-tags=all&from-date=2017-03-01&to-date=2017-07-01&order-by=newest&api-key=934fc87f-0792-4733-b5ed-59d5aa136db0&page=1
status ok
total 29336
startIndex 1
pageSize 50
pages 587
orderBy newest
---- RUNTIME STATUS ---- 
(1/587) in 0:00:00.971094s
(2/587) in 0:00:01.384066s
(3/587) in 0:00:01.821451s
(4/587) in 0:00:01.378031s
(5/587) in 0:00:01.054811s
(6/587) in 0:00:00.722367s
(7/587) in 0:00:00.705636s
(8/587) in 0:00:01.649562s
(9/587) in 0:00:02.515749s
(10/587) in 0:00:01.133337s
(11/587) in 0:00:01.161463s
(12/587) in 0:00:00.839989s
(13/587) in 0:00:00.830525s
(14/587) in 0:00:00.744745s
(15/587) in 0:00:01.018530s
(16/587) in 0:00:00.835961s
(17/587) in 0:00:00.740528s
(18/587) in 0:00:01.003446s
(19/587) in 0:00:02.991966s
(20/587) in 0:00:01.637009s
(21/587) in 0:00:00.982571s
(22/587) in 0:00:00.875486s
(23/587) in 0:00:00.822756s
(24/587) in 0:00:00.981257s
(25/587) i

In [11]:
@log_step
def init_pipeline(df):
    return df.copy()

@log_step
def unfold_columns(df):
    dict_cols = ["fields"] 
    for col in dict_cols:
        new_df = pd.DataFrame()
        new_df[col] = df[col]#.apply(lambda x: eval(x))
        add_cols_df = pd.json_normalize(new_df[col])
        df = pd.concat([df, add_cols_df], axis=1)
    
    # Tags extraction
    
    #df["tags"] = df["tags"].apply(lambda x: eval(x))
    df['tagWebTitle'] = df['tags'].map(lambda x:[i['webTitle'] for i in x])
    df['tagId'] = df['tags'].map(lambda x:[i['id'] for i in x])
    df = df.drop(columns="tags")
    return df.drop(columns=dict_cols)
@log_step
def drop_columns(df):
    df = df.drop(columns=[
        "body",
        "webUrl",
        "apiUrl",
        "sectionId",
        "pillarId",
        "showAffiliateLinks",
        "bylineHtml",
        "standfirst",
        "shouldHideReaderRevenue",
        "isInappropriateForSponsorship",
        "shortUrl",
        "isPremoderated",
        "charCount",
        "shouldHideAdverts",
        "showInRelatedContent",
        "lang",
        "main",
        "thumbnail"
        
    ])
    return df

@log_step
def create_csv(df):
    df.to_csv(f"../../data/partial/articles_partial_{iteration}_1.csv.zip", compression="zip")
    return df

In [12]:
articles = (
    init_df
        .pipe(init_pipeline)
        .pipe(unfold_columns)
        .pipe(drop_columns)
        .pipe(create_csv)
)

init_pipeline:
 shape=(29286, 13) took 0:00:00.007243s

unfold_columns:
 shape=(29286, 49) took 0:00:02.307728s

drop_columns:
 shape=(29286, 31) took 0:00:00.016328s

create_csv:
 shape=(29286, 31) took 0:00:18.346217s

